# Create NYC TLC Parquet files

There is New York City Taxi and Limousine Commission (TLC) Trip Record Data stored in S3, [see here](https://registry.opendata.aws/nyc-tlc-trip-records-pds/).

This notebook reads in the CSV data and writes out Parquet files that are easier and more performant to work with.

In [1]:
import coiled
import dask
import dask.dataframe as dd
import pandas as pd

In [2]:
cluster = coiled.Cluster(name="powers-tlc", n_workers=10)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/rich/live.py:22
1: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-matthew24-99557-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-99557-firewall -> coiled-dask-matthew24-99557-firewall]
Created FW rules: coiled-dask-matthew24-99557-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-99557-cluster-firewall -> coiled-dask-matthew24-99557-cluster-firewall]
Created scheduler VM: coiled-dask-matthew24-99557-scheduler (type: t3.medium, ip: ['18.215.186.126'])


In [3]:
client = dask.distributed.Client(cluster)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/client.py:1131: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| dask        | 2021.11.2 | 2022.01.0 | 2022.01.0 |
| distributed | 2021.11.2 | 2022.01.0 | 2022.01.0 |
| numpy       | 1.22.0    | 1.21.5    | 1.21.5    |
+-------------+-----------+-----------+-----------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


## 2009 data create

In [67]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2009-*.csv",
    parse_dates=["Trip_Pickup_DateTime", "Trip_Dropoff_DateTime"],
    dtype={
        "Tolls_Amt": "float64",
        "store_and_forward": "object",
    },
)

In [68]:
import math


def rate_code_to_one(something):
    if math.isnan(something):
        return 1
    else:
        return something

In [69]:
ddf = ddf.assign(
    Rate_Code=ddf.Rate_Code.apply(rate_code_to_one, meta=("float64", "int64"))
)

In [70]:
# vendor_id                     object
# pickup_datetime       datetime64[ns]
# dropoff_datetime      datetime64[ns]
# passenger_count                int64
# trip_distance                float64
# pickup_longitude             float64
# pickup_latitude              float64
# rate_code                      int64
# store_and_fwd_flag            object
# dropoff_longitude            float64
# dropoff_latitude             float64
# payment_type                  object
# fare_amount                  float64
# surcharge                    float64
# mta_tax                      float64
# tip_amount                   float64
# tolls_amount                 float64
# total_amount                 float64
# dtype: object
ddf.dtypes

vendor_name                      object
Trip_Pickup_DateTime     datetime64[ns]
Trip_Dropoff_DateTime    datetime64[ns]
Passenger_Count                   int64
Trip_Distance                   float64
Start_Lon                       float64
Start_Lat                       float64
Rate_Code                         int64
store_and_forward                object
End_Lon                         float64
End_Lat                         float64
Payment_Type                     object
Fare_Amt                        float64
surcharge                       float64
mta_tax                         float64
Tip_Amt                         float64
Tolls_Amt                       float64
Total_Amt                       float64
dtype: object

In [71]:
ddf = ddf.rename(
    columns={
        "vendor_name": "vendor_id",
        "Trip_Pickup_DateTime": "pickup_datetime",
        "Trip_Dropoff_DateTime": "dropoff_datetime",
        "Passenger_Count": "passenger_count",
        "Trip_Distance": "trip_distance",
        "Start_Lon": "pickup_longitude",
        "Start_Lat": "pickup_latitude",
        "Rate_Code": "rate_code",
        "store_and_forward": "store_and_fwd_flag",
        "End_Lon": "dropoff_longitude",
        "End_Lat": "dropoff_latitude",
        "Payment_Type": "payment_type",
        "Fare_Amt": "fare_amount",
        "surcharge": "surcharge",
        "mta_tax": "mta_tax",
        "Tip_Amt": "tip_amount",
        "Tolls_Amt": "tolls_amount",
        "Total_Amt": "total_amount",
    }
)

In [72]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [73]:
len(ddf.dtypes)

18

In [74]:
actual_dtypes_2010 == ddf.dtypes

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

In [75]:
ddf.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2009-01-04 02:52:00,2009-01-04 03:02:00,1,2.63,-73.991957,40.721567,1,NaN,-73.993803,40.695922,CASH,8.9,0.5,NaN,0.00,0.0,9.40
1,VTS,2009-01-04 03:31:00,2009-01-04 03:38:00,3,4.55,-73.982102,40.736290,1,NaN,-73.955850,40.768030,Credit,12.1,0.5,NaN,2.00,0.0,14.60
2,VTS,2009-01-03 15:43:00,2009-01-03 15:57:00,5,10.35,-74.002587,40.739748,1,NaN,-73.869983,40.770225,Credit,23.7,0.0,NaN,4.74,0.0,28.44
3,DDS,2009-01-01 20:52:58,2009-01-01 21:14:00,1,5.00,-73.974267,40.790955,1,NaN,-73.996558,40.731849,CREDIT,14.9,0.5,NaN,3.05,0.0,18.45
4,DDS,2009-01-24 16:18:23,2009-01-24 16:24:56,1,0.40,-74.001580,40.719382,1,NaN,-74.008378,40.720350,CASH,3.7,0.0,NaN,0.00,0.0,3.70


In [76]:
ddf.npartitions

490

In [77]:
ddf.known_divisions

False

In [78]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc/2009",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

## 2009 data query

In [45]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2009",
    engine="pyarrow",
)

In [46]:
dtypes_2009 = ddf.dtypes

In [49]:
dtypes_2009

vendor_name                      string
Trip_Pickup_DateTime     datetime64[ns]
Trip_Dropoff_DateTime    datetime64[ns]
Passenger_Count                   int64
Trip_Distance                   float64
Start_Lon                       float64
Start_Lat                       float64
Rate_Code                       float64
store_and_forward               float64
End_Lon                         float64
End_Lat                         float64
Payment_Type                     string
Fare_Amt                        float64
surcharge                       float64
mta_tax                         float64
Tip_Amt                         float64
Tolls_Amt                       float64
Total_Amt                       float64
dtype: object

In [38]:
len(ddf)

170896055

In [42]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2009", engine="pyarrow", columns=["Fare_Amt"]
)

In [44]:
%%time
ddf.Fare_Amt.mean().compute()

CPU times: user 124 ms, sys: 7.65 ms, total: 131 ms
Wall time: 4.79 s


9.905162372589585

In [ ]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2009",
    engine="pyarrow",
)

In [ ]:
ddf.dtypes

## 2010 data create

In [45]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2010-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    on_bad_lines="skip",
    dtype={
        "tolls_amount": "float64",
        "store_and_fwd_flag": "object",
    },
)

In [46]:
ddf.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,VTS,2010-01-26 07:41:00,2010-01-26 07:45:00,1,0.75,-73.956778,40.767750,1,NaN,-73.965957,40.765232,CAS,4.5,0.0,0.5,0.00,0.0,5.00
1,DDS,2010-01-30 23:31:00,2010-01-30 23:46:12,1,5.90,-73.996118,40.763932,1,NaN,-73.981512,40.741193,CAS,15.3,0.5,0.5,0.00,0.0,16.30
2,DDS,2010-01-18 20:22:20,2010-01-18 20:38:12,1,4.00,-73.979673,40.783790,1,NaN,-73.917852,40.878560,CAS,11.7,0.5,0.5,0.00,0.0,12.70
3,VTS,2010-01-09 01:18:00,2010-01-09 01:35:00,2,4.70,-73.977922,40.763997,1,NaN,-73.923908,40.759725,CAS,13.3,0.5,0.5,0.00,0.0,14.30
4,CMT,2010-01-18 19:10:14,2010-01-18 19:17:07,1,0.60,-73.990924,40.734682,1,0,-73.995511,40.739088,Cre,5.3,0.0,0.5,0.87,0.0,6.67


In [5]:
dtypes2010 = ddf.dtypes

In [6]:
dtypes2010

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [8]:
ddf.to_parquet(
    "s3://coiled-datasets/nyc-tlc/2010",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

In [18]:
ddf = dd.read_parquet("s3://coiled-datasets/nyc-tlc/2010", engine="pyarrow")

In [19]:
actual_dtypes_2010 = ddf.dtypes

## 2011 data create

In [14]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2011-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    dtype={
        "tip_amount": "float64",
        "tolls_amount": "float64",
        # "vendor_name": "string[pyarrow]",
        # "Payment_Type": "string[pyarrow]",
    },
)

In [15]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [17]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc/2011",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

In [12]:
ddf = dd.read_parquet("s3://coiled-datasets/nyc-tlc/2011", engine="pyarrow")

In [13]:
actual_dtypes_2011 = ddf.dtypes

In [15]:
type(actual_dtypes_2011)

pandas.core.series.Series

In [21]:
actual_dtypes_2011

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [20]:
actual_dtypes_2011 == actual_dtypes_2010

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

## 2012 Data Create

In [22]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2012-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    dtype={
        "tip_amount": "float64",
        "tolls_amount": "float64",
    },
)

In [23]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [24]:
ddf.dtypes == actual_dtypes_2010

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

In [25]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc/2012",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

## 2013 Data Create

In [26]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2013-*.csv",
    parse_dates=["pickup_datetime", "dropoff_datetime"],
    dtype={
        "tip_amount": "float64",
        "tolls_amount": "float64",
    },
)

In [27]:
ddf.dtypes == actual_dtypes_2010

vendor_id             True
pickup_datetime       True
dropoff_datetime      True
passenger_count       True
trip_distance         True
pickup_longitude      True
pickup_latitude       True
rate_code             True
store_and_fwd_flag    True
dropoff_longitude     True
dropoff_latitude      True
payment_type          True
fare_amount           True
surcharge             True
mta_tax               True
tip_amount            True
tolls_amount          True
total_amount          True
dtype: bool

In [28]:
len(ddf.dtypes)

18

In [29]:
ddf.dtypes

vendor_id                     object
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
trip_distance                float64
pickup_longitude             float64
pickup_latitude              float64
rate_code                      int64
store_and_fwd_flag            object
dropoff_longitude            float64
dropoff_latitude             float64
payment_type                  object
fare_amount                  float64
surcharge                    float64
mta_tax                      float64
tip_amount                   float64
tolls_amount                 float64
total_amount                 float64
dtype: object

In [30]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc/2013",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

## 2019 Data Create

In [4]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2019-*.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        "RatecodeID": "float64",
        "VendorID": "float64",
        "passenger_count": "float64",
        "payment_type": "object",
    },
)

In [6]:
ddf.dtypes

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                     object
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

In [16]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc/2019",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=False,
)

[None]

In [7]:
ddf.repartition(partition_size="100MB").to_parquet(
    "s3://coiled-datasets/nyc-tlc-with-metadata/2019",
    engine="pyarrow",
    compression="snappy",
    write_metadata_file=True,
)

[None]

## 2020 Data

In [17]:
ddf = dd.read_csv(
    "s3://nyc-tlc/trip data/yellow_tripdata_2020-*.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    dtype={
        "RatecodeID": "float64",
        "VendorID": "float64",
        "passenger_count": "float64",
        "payment_type": "object",
    },
)

In [18]:
ddf.dtypes

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                     object
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                      int64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
dtype: object

## Data queries

In [86]:
ddf2009 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2009",
    engine="pyarrow",
)

In [87]:
len(ddf2009)

OSError: [Errno 22] Bad Request

In [80]:
ddf2010 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2010",
    engine="pyarrow",
)

In [81]:
ddf2011 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2011",
    engine="pyarrow",
)

In [82]:
ddf2012 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2012",
    engine="pyarrow",
)

In [83]:
ddf2013 = dd.read_parquet(
    "s3://coiled-datasets/nyc-tlc/2013",
    engine="pyarrow",
)

In [84]:
ddf = dd.concat([ddf2009, ddf2010, ddf2011, ddf2012, ddf2013])

In [85]:
len(ddf)

PermissionError: The provided token has expired.